In [1]:
import pandas as pd

# Ler o arquivo Excel
df = pd.read_excel("systematik.xlsx")
df

,Level1,Level2,Level3,Level4,Level5,Level6
0,Einzelne Länder,Afrika,NaN,NaN,NaN,NaN
1,Einzelne Länder,Asien,Japan,OA/Jap: I,NaN,NaN
2,Einzelne Länder,Asien,Japan,OA/Jap: II,NaN,NaN
3,Einzelne Länder,Australien,NaN,NaN,NaN,NaN
4,Einzelne Länder,Mittelamerika,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
363,Europäische Union,EG: XVII Zivilrecht.,EG: XVII A Bürgerliches Recht.,https://aleph.mpg.de/F?func=find-b&local_base=...,NaN,NaN
364,Europäische Union,EG: XVII Zivilrecht.,EG: XVII B Handelsrecht.,https://aleph.mpg.de/F?func=find-b&local_base=...,NaN,NaN
365,Europäische Union,EG: XVII Zivilrecht.,EG: XVII C Gesellschaftsrecht. EWIV.,https://aleph.mpg.de/F?func=find-b&local_base=...,NaN,NaN
366,Europäische Union,EG: XVII Zivilrecht.,EG: XVII D Urheberrecht.,https://aleph.mpg.de/F?func=find-b&local_base=...,NaN,NaN


In [2]:
class Node:
    def __init__(self, name, url=None):
        self.name = name
        self.url = url
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

def build_tree_from_df(df):
    roots = {}
    for _, row in df.iterrows():
        # Identifica os níveis preenchidos (não nulos)
        levels = [row[f'Level{i}'] for i in range(1, df.shape[1] + 1) if pd.notna(row[f'Level{i}'])]

        # Considera o último nível como sendo a URL, se aplicável
        url = levels.pop() if levels and isinstance(levels[-1], str) and levels[-1].startswith(('http://', 'https://')) else None
        
        # Filtra os níveis indesejados ("BC" e "B6")
        if any(level in ['BC', 'B6'] for level in levels):
            continue
        
        current_node = None
        for i, level in enumerate(levels):
            if i == 0:
                if level not in roots:
                    roots[level] = Node(level)
                current_node = roots[level]
            else:
                child_node = next((child for child in current_node.children if child.name == level), None)
                if not child_node:
                    child_node = Node(level)
                    current_node.add_child(child_node)
                current_node = child_node
        
        # Associa a URL ao último nó da linha, se houver uma URL
        if url:
            current_node.url = url

    return list(roots.values())

def generate_html(node):
    # Define a cor e o estilo do link se houver URL
    if node.url:
        name_html = f'<a href="{node.url}" target="_blank" style="color: #007c6c; text-decoration: underline;">{node.name}</a>'
    else:
        name_html = node.name

    if not node.children:
        return f'<li>{name_html}</li>'
    
    children_html = ''.join(generate_html(child) for child in node.children)
    return f'''
    <li>
        <span class="caret">{name_html}</span>
        <ul class="nested">
            {children_html}
        </ul>
    </li>
    '''

def generate_forest_html(forest):
    forest_html = ''.join(generate_html(root) for root in forest)
    return f'''

<!DOCTYPE html>
<html lang="de">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MPILs Systematik</title>
    <style>
        ul {{
            list-style-type: none; /* Remove bullet points */
        }}
        li {{
            color: black; /* Default text color is black */
        }}
        li:hover {{
            color: #007c6c; /* Minerva green color on hover */
        }}
        a {{
            color: inherit; /* Inherit the text color */
            text-decoration: none; /* Remove underline by default */
        }}
        a:hover {{
            color: #007c6c; /* Minerva green color on hover */
            text-decoration: underline; /* Underline on hover */
        }}
        .nested {{
            display: none;
        }}
        .active {{
            display: block;
        }}
        .caret {{
            cursor: pointer;
            user-select: none;
            font-weight: bold; /* Highlight caret items */
        }}

    </style>
</head>
<body>
    <h2>MPILs Systematik</h2>
    <ul id="myMenu">
        {forest_html}
    </ul>
    <script>
        document.addEventListener('DOMContentLoaded', function() {{
            var toggler = document.getElementsByClassName("caret");
            for (var i = 0; i < toggler.length; i++) {{
                toggler[i].addEventListener("click", function() {{
                    this.parentElement.querySelector(".nested").classList.toggle("active");
                    this.classList.toggle("caret-down");
                }});
            }}
        }});
    </script>
</body>
</html>
    '''

# Ler o arquivo Excel
df = pd.read_excel('systematik.xlsx')

# Construir a árvore a partir do DataFrame
forest = build_tree_from_df(df)

# Gerar conteúdo HTML
html_content = generate_forest_html(forest)

# Salvar o conteúdo HTML em um arquivo
with open('Test_7.html', 'w') as file:
    file.write(html_content)

print("HTML file 'Test_7.html' has been generated.")


HTML file 'Test_7.html' has been generated.
